In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import skimage
import tqdm
import skimage.morphology
import ipywidgets as widgets
import json
import tqdm
import io

from ipywidgets import interact, interact_manual
from scipy.optimize import basinhopping
from scipy import optimize

from pathlib import Path
from PIL import Image as PILImage

from IPython.display import clear_output, Image

In [ ]:
data_path = Path('/data/Datasets/usg-kaggle/train/')

In [ ]:
def get_bouncing_coordinate(coordinate_value, limit):
    if coordinate_value >= 0: 
        if (coordinate_value // limit) % 2 == 0:
            return coordinate_value % limit
        return limit - (coordinate_value % limit) - 1
    else:
        if (abs(coordinate_value) // limit) % 2 == 0:
            return limit - (coordinate_value % limit) -1
        return coordinate_value % limit

In [ ]:
def get_pixel_values(center, angle, image):
    x_c, y_c = center
    a = np.tan(np.deg2rad(angle))
    b = y_c - (a * x_c)
    mask = np.zeros_like(image)
    x1 = min(max(-b / (a + 1e-8), 0), image.shape[1])
    y1 = int(a * x1 + b)
    x1 = int(x1)
    
    start_point = np.asarray([[y1, x1]])
    
    x2 = min(max((image.shape[1] - b) / (a + 1e-8), 0), image.shape[1])
    y2 = int(a * x2 + b)
    x2 = int(x2)
    
    mask = np.zeros_like(image)
    cv2.line(mask, (x1, y1), (x2, y2), 1, 1, cv2.LINE_AA)
    points = np.stack(np.where(mask == 1), axis=-1)
    values = []
    kernel_size = 3
    for y, x in points:
        a_sum = 0
        for of_y in range(-kernel_size // 2, kernel_size // 2 + 1):
            for of_x in range(-kernel_size // 2, kernel_size // 2 + 1):
                a_sum += image[
                    get_bouncing_coordinate(y + of_y, image.shape[0]), 
                    get_bouncing_coordinate(x + of_x, image.shape[1])
                ]
        values.append(a_sum)
    
    distances = np.sqrt(np.sum((points - start_point) ** 2, axis=1))
    return values, distances, mask

In [ ]:
directories = [path for path in data_path.glob('*') if path.is_dir()]
dir_widget = widgets.Dropdown(
    options=directories,
    index=0,
    description="Directory:"
)

files = list([path for path in Path(dir_widget.value).rglob('lower.png') if not path.name.startswith('.')])
files_widget = widgets.Dropdown(
    options=files,
    index=0,
    description="File:"
)


with files_widget.hold_trait_notifications():
    files = list([path for path in Path(dir_widget.value).rglob('lower.png') if not path.name.startswith('.')])
    files = set(files)
    if len(files.difference(set(files_widget.options))) > 0:
        files_widget.options = files

In [ ]:
angle_widget = widgets.IntSlider(
    min=0,
    max=359,
    value=90,
    description="Angle:"
)


@interact(a_dir=dir_widget, a_file=files_widget, angle=angle_widget)
def get_img(a_dir, a_file, angle):
    with files_widget.hold_trait_notifications():
        files = list([path for path in Path(dir_widget.value).rglob('lower.png') if not path.name.startswith('.')])
        files = set(files)
        if len(files.difference(set(files_widget.options))) > 0:
            files_widget.options = files
        
    img_file = cv2.imread(a_file.as_posix(), cv2.IMREAD_GRAYSCALE)
    img_file = ((img_file.astype('float32') / 255) ** 8)
    img_file = ((img_file - img_file.min()) / (img_file.max() - img_file.min()) * 255).astype('uint8')
    coords = json.loads((a_file.parent / "coordinates.json").read_text())
    circle_coords = coords["circle"]
    
    mask = np.zeros_like(img_file)
    radia = []
    sums = []
    circle_center = circle_coords["x"], circle_coords["y"]
    for i in range(5, 50):
        cv2.circle(mask, circle_center, i, 255, 1)
        sums.append(img_file[mask == 255].sum() / (2 * np.pi * i))
        radia.append(i)
        mask[:] = 0
        
    gradients = np.diff(sums)
    radia = radia[:-1]
    ascendence = radia[np.argmax(gradients)]
    descendence = radia[np.argmin(gradients)]
    
    orig = img_file.copy()
    img_file = cv2.medianBlur(img_file, 3)
    values, points, mask = get_pixel_values(circle_center, angle, img_file)
    fig, ax = plt.subplots(1, 3, figsize=(18, 4))
    ax[0].plot(radia, gradients)
    ax[1].plot(points, values)
    ax[2].plot(points[1:], np.diff(values))
    
    fig, ax = plt.subplots(1, 2, figsize=(18.6 * 2, 18.6), sharey=True)
    ax[0].imshow(orig, cmap='gray')
    ax[1].imshow(mask, cmap="gray")
    fig.tight_layout()
    
    le_image = np.abs(orig.astype(np.float32) - img_file.astype(np.float32))
    le_image = (le_image - le_image.min()) / (le_image.max() - le_image.min())
    le_image[le_image > np.percentile(le_image, 98)] = 1
    le_image[le_image <= np.percentile(le_image, 98)] = 0
    
    kernel = np.ones((1,1),np.uint8)
    le_image = cv2.medianBlur(le_image, 3)
    
    plt.figure(figsize=(18.6, 18.6))
    plt.imshow(le_image)

In [ ]:
x = widgets.IntSlider(0, 0, 1049)
y = widgets.IntSlider(0, 0, 325)
width = widgets.IntSlider(1, 1, 1050)
height = widgets.IntSlider(1, 1, 326)

@interact(a_dir=dir_widget, a_file=files_widget, x=x, y=y, width=width, height=height)
def estimate_noise(a_dir, a_file, x, y, width, height):
    width = min(width + x, 1050) - x
    height = min(height + y, 326) - y
    img_file = cv2.imread(a_file.as_posix(), 0)
    
    fig, ax = plt.subplots(2, 1, figsize=(8, 8))
    ax[0].imshow(img_file)
    ax[1].imshow(img_file[y:y+height, x:x+width])
    
    sample = img_file[y:y+height, x:x+width]
    print(sample.mean(), sample.std())
    

In [ ]:
from scipy.fftpack import dct, idct

def fit_img(img: np.ndarray) -> np.ndarray:
    def get_min_max_cord(img, axis):
        img = img.astype(np.float32)
        line = img.sum(axis=axis)
        line[line > 0] = 1
        grad = np.diff(line)
        a_min = np.argmax(grad)
        a_max = np.argmin(grad)
        return a_min, a_max
    
    y_min, y_max = get_min_max_cord(img, 1)
    x_min, x_max = get_min_max_cord(img, 0)
    return img[y_min:y_max, x_min:x_max]
    

def low_pass_filter(img_block: np.ndarray, a: int = 8) -> np.ndarray:
    dct_block = cv2.dct(img_block)
    block_size = dct_block.shape[0]
    assert 0 < a < block_size - 1, "Parameter 'a' should be 0 < a < block_size " \
        "for block size equal to {}, received {} instead".format(block_size, a)
    
    u, v = np.meshgrid(np.arange(0, block_size), np.arange(0, block_size))
    u_minus_v = u - v
    where_less_than_a = u_minus_v < a
    dct_block[~where_less_than_a] = 0

    restored = cv2.dct(dct_block, cv2.DCT_INVERSE)
    
    return restored

def gaussian(img_block: np.ndarray, **kwargs) -> np.ndarray:
    return cv2.blur(img_block, (7, 7))
    
def get_biggest_component(img):
    binary = (img > 0).astype(np.uint8) * 255
    connectivity = 8
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(binary, connectivity, cv2.CV_16U)
    areas = stats[:, cv2.CC_STAT_AREA]
    label_with_largest_area = np.argmax(areas)
    indices = labels == np.argsort(-areas)[1]
    img[~indices] = 0
    return img
    
def get_clean_and_mask_images(img):
    img = get_biggest_component(img)
    img = fit_img(img)
    
    img_min, img_max = img.min(), img.max()
    fl_img = img.astype(np.float32)
    
    hor_grad = fl_img[1:] - fl_img[:-1] 
    ver_grad = fl_img[:, 1:] - fl_img[:, :-1]
    
    hor_grad = hor_grad[:, 1:]
    ver_grad = ver_grad[1:]
    
    grad = hor_grad + ver_grad
    grad = (grad - grad.min()) / (grad.max() - grad.min())
    grad = grad - grad.mean()
    grad = np.clip(grad, 0, 1) 
    
    offset_h = 50
    offset_img = img[offset_h:]
    percentile = np.percentile(grad, q=95)    
    masked = (grad > percentile).astype(np.uint8)
        
    kernel = np.ones((3, 3), np.uint8)
    masked = cv2.morphologyEx(masked, cv2.MORPH_DILATE, kernel, iterations=1)
    masked = np.pad(masked, [[1, 0], [1, 0]], constant_values=0, mode="constant")

    
    connectivity = 4
    num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(masked, connectivity, cv2.CV_16U)
    areas = stats[:, cv2.CC_STAT_AREA]
    for i, area in enumerate(areas):
        if area < 256:
            indices = labels == i
            masked[indices] = 0
    
    
    
    removed = img * (1 - masked)
    output_img = np.zeros_like(img).astype(np.float32)
    
    block_size = 64
    step = block_size // 2
    output_img = cv2.inpaint(img, masked, 5, cv2.INPAINT_TELEA)
    return output_img, masked
    

@interact(a_dir=dir_widget, 
          a_file=files_widget)
def wut(a_dir, a_file):
    print(a_file)
    img = cv2.imread(a_file.as_posix(), 0)
    output_img, masked = get_clean_and_mask_images(img)
 
    
    fig, ax = plt.subplots(2, 1, figsize=(32, 32))
    ax[0].imshow(masked, cmap='gray')
    ax[1].imshow(output_img, cmap='gray')

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim

class Unet(nn.Module):
    def __init__(self, init_channels, out_channels):
        super().__init__()
        
        self.n_u = self.n_d = [16, 16, 16, 16 ,16]
        self.k_u = self.k_d = [3, 3, 3, 3, 3]
        self.n_s = [4, 4, 4, 4, 4]
        self.k_s = [1, 1, 1, 1, 1]
        self.init_channels = init_channels
        self.out_channels = out_channels
        
        self.coding_layers = nn.ModuleList()
        self.decoding_layers = nn.ModuleList()
        self.skip_layers = nn.ModuleList()
        
        next_filters = self.init_channels
        decoder_input_filters = []
        
        for i in range(len(self.n_u)):
            self.coding_layers.append(
                self._encoder_block(next_filters, self.n_u[i], self.k_u[i])
            )
            self.skip_layers.append(
                self._skip_block(self.n_u[i], self.n_s[i], self.k_s[i])
            )
            
            next_filters = self.n_u[i]
            decoder_input_filters.append(
                self.n_u[i] + self.n_s[i]
            )
            
        decoder_input_filters = decoder_input_filters[::-1]
        for i in range(len(self.n_d)):
            self.decoding_layers.append(
                self._decoder_block(decoder_input_filters[i], self.n_d[i], self.k_d[i])
            )
            
        self.out_layer = nn.Sequential(
            nn.Conv2d(self.n_d[-1], self.out_channels, 1, bias=True),
            nn.Sigmoid()
        )
        
        
    def forward(self, x):
        skip_results = []
        for i, layer in enumerate(self.coding_layers):
            x = layer(x)
            skip_results.append(
                self.skip_layers[i](x)
            )
        
        skip_results = skip_results[::-1]
        
        for i, layer in enumerate(self.decoding_layers):
            without_skip = x
            x = torch.cat((x, skip_results[i]), dim=1)
            x = layer(x)
        
        return self.out_layer(x)
        
        
    def _encoder_block(self, in_channels, filters, kernel_size, dilation=1):
        layers = nn.Sequential(
            nn.Conv2d(in_channels, filters, kernel_size, 2, kernel_size // 2, bias=False, padding_mode="reflection", dilation=dilation),
            nn.InstanceNorm2d(filters),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(filters, filters, kernel_size, 1, kernel_size // 2, bias=False, padding_mode="reflection", dilation=dilation),
            nn.InstanceNorm2d(filters),
            nn.LeakyReLU(inplace=True)
        )
        return layers
    
    def _decoder_block(self, in_channels, filters, kernel_size, dilation=1):
        layers = nn.Sequential(
            nn.InstanceNorm2d(in_channels),
            nn.Conv2d(in_channels, filters, kernel_size, 1, kernel_size // 2, bias=False, padding_mode="reflection", dilation=dilation),
            nn.InstanceNorm2d(filters),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(filters, filters, 1, 1, 0, bias=False, dilation=dilation),
            nn.InstanceNorm2d(filters),
            nn.LeakyReLU(inplace=True),
            nn.UpsamplingBilinear2d(scale_factor=2)
        )
        return layers
    
    def _skip_block(self, in_channels, filters, kernel_size):
        layers = nn.Sequential(
            nn.Conv2d(in_channels, filters, kernel_size, 1, 0, bias=False),
            nn.InstanceNorm2d(filters),
            nn.LeakyReLU(inplace=True)
        )
        return layers

In [ ]:
def reconstruction_loss(y_pred, y_true, mask):
    diff = (y_pred - y_true) * mask
    loss = diff.pow(2).mean()
    return loss

def match_dims_to_be_divisible(an_img):
    h, w = an_img.shape[0], an_img.shape[1]
    
    new_h = (h // 32 + 1) * 32
    new_w = (w // 32 + 1) * 32
    return new_h, new_w

In [ ]:
a_model = Unet(128, 1)

an_img = cv2.imread("/data/Datasets/usg-kaggle/train/1/497/lower.png", 0)
orig_img = fit_img(get_biggest_component(an_img))
an_img, mask = get_clean_and_mask_images(ori)
orig_h, orig_w = an_img.shape

new_h, new_w = match_dims_to_be_divisible(an_img)
an_img = cv2.resize(an_img, (new_w, new_h))
mask = 1 - cv2.resize(mask, (new_w, new_h))
orig_img = cv2.resize(orig_img, (new_w, new_h))

an_img = an_img.astype(np.float32) / 255
mask = mask.astype(np.float32)
an_img = an_img * mask

h, w = an_img.shape

an_img = torch.from_numpy(an_img).float().unsqueeze(0).unsqueeze(0)
mask = torch.from_numpy(mask).float().unsqueeze(0).unsqueeze(0)
optimised_image = torch.randn((1, 128, h, w)).float()
optimised_image.uniform_(0, 1)
optimised_image.requires_grad = True

num_iters = 2000
lr = 0.01


optimiser = optim.Adam(a_model.parameters(), lr=lr)
out_widget = widgets.Output()
display(out_widget)
print()
for i in range(num_iters):
    with out_widget:
        print("\rIter: {} / {}".format(i + 1, num_iters), end='')
    optimiser.zero_grad()
    prediction = a_model(optimised_image)
    loss = reconstruction_loss(prediction, an_img, mask)
    loss.backward()
    optimiser.step()
    
    if i % 10 == 0:
        pred_img = prediction[0, 0].detach().cpu().numpy() * 255
        pred_img = np.clip(pred_img, 0, 255).astype(np.uint8)
        pred_img = np.concatenate([pred_img, orig_img], axis=1)
        pil_iamge = PILImage.fromarray(pred_img)
        img_byte_array = io.BytesIO()
        pil_iamge.save(img_byte_array, format="PNG")
        img_byte_array = img_byte_array.getvalue()
        print()
        clear_output()
        display(out_widget)
        display(Image(img_byte_array))

In [ ]:

an_img = cv2.imread("/data/Datasets/usg-kaggle/train/1/156/lower.png", 0)
plt.figure(figsize=(12, 6))
plt.imshow(an_img, cmap='gray')
plt.show()